# LangGraph 100本ノック

このJupyter Notebookは、LangGraphの基礎から応用までを体系的に学ぶための100問の演習問題とその解答、解説を提供します。LangGraphを用いた複雑なエージェントワークフローの構築スキルを習得しましょう。

## 目次

*   **第1章: グラフの基本要素 (問題 001-020):** `StateGraph`, `State`, ノード, エッジの定義。最もシンプルなグラフの構築と実行。
*   **第2章: グラフの制御フロー (問題 021-040):** 条件付きエッジによる分岐、自己修正ループ、エラーハンドリング、人間による介入(`Interrupt`)。
*   **第3章: ツールを使うシングルエージェント (問題 041-060):** `ToolNode`の活用、ReAct型エージェント、計画と実行(Plan-and-Execute)型エージェントの構築。
*   **第4章: マルチエージェント・ワークフロー (問題 061-090):** スーパーバイザー型によるタスク割り振り、複数エージェントによる対話シミュレーション、階層型エージェント。
*   **第5章: 発展的なグラフ技術 (問題 091-100):** Agent Swarm、永続化(`Checkpointer`)による状態保存と再開、ストリーミング、カスタムエージェント。

In [ ]:
# === ライブラリのインストール ===
# ご利用になるLLMプロバイダーに応じて、以下のコメントを解除して実行してください。
# !%pip install -qU langchain langchain_openai langchain_google_vertexai langchain_anthropic langchain_aws boto3

# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google", "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

# === APIキー/環境変数の設定 ===
import os
from google.colab import userdata # Google Colab を利用する場合

# --- OpenAI ---
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# または Colab の場合
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- Azure OpenAI ---
# os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_AZURE_OPENAI_API_KEY"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_AZURE_OPENAI_ENDPOINT"
# os.environ["OPENAI_API_VERSION"] = "YOUR_OPENAI_API_VERSION" # 例: "2023-07-01-preview"
# または Colab の場合
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")
# os.environ["OPENAI_API_VERSION"] = userdata.get("OPENAI_API_VERSION")


# --- Google Cloud Vertex AI (Gemini) ---
# Google CloudプロジェクトIDを設定
# PROJECT_ID = "YOUR_GOOGLE_CLOUD_PROJECT_ID"
# LOCATION = "YOUR_GOOGLE_CLOUD_LOCATION" # 例: "us-central1"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_YOUR_SERVICE_ACCOUNT_KEY_JSON"
#
# from google.colab import auth as google_auth # Google Colab を利用する場合
# google_auth.authenticate_user()
#
# if LLM_PROVIDER == "google" and not os.getenv("GOOGLE_CLOUD_PROJECT") and not PROJECT_ID: # PROJECT_IDが未定義の場合も考慮
#     # Google Colab環境でPROJECT_IDが設定されていない場合、ユーザーに設定を促すか、デフォルトのプロジェクトを使用するなどの処理
#     try:
#         PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID") # Colabのシークレットから読み込む試み
#         if not PROJECT_ID:
#             print("警告: Google Cloud Project ID が設定されていません。Vertex AI を利用する場合は設定が必要です。")
#     except Exception:
#          print("警告: Google Cloud Project ID が設定されていません。Vertex AI を利用する場合は設定が必要です。")
#
# if LLM_PROVIDER == "google" and not PROJECT_ID and not os.getenv("GOOGLE_CLOUD_PROJECT"):
#     raise ValueError("Google Cloud Project ID is not set. Please set the PROJECT_ID variable or GOOGLE_CLOUD_PROJECT environment variable.")


# --- Anthropic (Claude) ---
# os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"
# または Colab の場合
# os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

# --- Amazon Bedrock (Claude) ---
# AWS認証情報を設定 (環境変数、IAMロール、または ~/.aws/credentials で設定)
# os.environ["AWS_ACCESS_KEY_ID"] = "YOUR_AWS_ACCESS_KEY_ID"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "YOUR_AWS_SECRET_ACCESS_KEY"
# os.environ["AWS_REGION_NAME"] = "YOUR_AWS_REGION_NAME" # 例: "us-east-1"
# または Colab の場合
# os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
# os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
# os.environ["AWS_REGION_NAME"] = userdata.get("AWS_REGION_NAME")


# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    # 利用可能なモデル名の例: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    # Azure OpenAI のデプロイ名を指定
    # AZURE_OPENAI_DEPLOYMENT_NAME = "YOUR_AZURE_OPENAI_DEPLOYMENT_NAME"
    # または環境変数 AZURE_OPENAI_DEPLOYMENT_NAME から取得
    # または Colab の場合
    # AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")
    azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    if not azure_deployment:
        try:
            azure_deployment = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")
        except Exception:
            pass # userdata が利用できない環境やキーが存在しない場合
    if not azure_deployment:
        raise ValueError("Azure OpenAI deployment name is not set. Please set the AZURE_OPENAI_DEPLOYMENT_NAME environment variable or define it in the notebook.")

    llm = AzureChatOpenAI(
        azure_deployment=azure_deployment,
        openai_api_version=os.environ.get("OPENAI_API_VERSION", "2023-07-01-preview"),
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # 利用可能なモデル名の例: "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-pro"
    # PROJECT_ID と LOCATION が正しく設定されていることを確認
    # LOCATION のデフォルト値を設定
    resolved_project_id = os.getenv("GOOGLE_CLOUD_PROJECT", PROJECT_ID if 'PROJECT_ID' in locals() else None)
    resolved_location = os.getenv("GOOGLE_CLOUD_LOCATION", LOCATION if 'LOCATION' in locals() else "us-central1")

    if not resolved_project_id:
        raise ValueError("Google Cloud Project ID is not set. Please set the PROJECT_ID variable or GOOGLE_CLOUD_PROJECT environment variable.")

    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=resolved_project_id, location=resolved_location)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    # 利用可能なモデル名の例: "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307"
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock
    # 利用可能なモデルIDの例:
    # Anthropic Claude: "anthropic.claude-3-sonnet-20240229-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"
    # Meta Llama 3: "meta.llama3-8b-instruct-v1:0"
    # Mistral: "mistral.mistral-large-2402-v1:0"
    # Cohere: "cohere.command-r-v1:0"
    resolved_region_name = os.getenv("AWS_REGION_NAME")
    if not resolved_region_name:
        try:
            resolved_region_name = userdata.get("AWS_REGION_NAME")
        except Exception:
            pass # userdata が利用できない環境やキーが存在しない場合
    # BedrockChatはregion_nameがNoneでも動作する場合があるため、必須とはしないが、指定を推奨

    llm = BedrockChat(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        region_name=resolved_region_name,
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
# print(f"LLM Client: {llm}") # 詳細なクライアント情報はデバッグ時以外は不要な場合もあるためコメントアウトも検討
if llm:
    print(f"LLM Client Type: {type(llm)}")
    if hasattr(llm, 'model_name'):
        print(f"LLM Model: {llm.model_name}")
    elif hasattr(llm, 'model'):
        print(f"LLM Model: {llm.model}")
    elif hasattr(llm, 'model_id'):
        print(f"LLM Model: {llm.model_id}")


### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

In [ ]:
# 解答欄001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("simple_node", simple_node)

# エントリポイントの設定
workflow.____("simple_node")

# 終了ポイントの設定
workflow.____("simple_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.____(inputs):
    print(s)

# 最終結果の確認
final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.stream(inputs):
    print(s)

# 最終結果の確認
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。
*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `app = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `app.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`app.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>

### ■ 問題002: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

In [ ]:
# 解答欄002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [____(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [____(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("uppercase", uppercase_node)
workflow.____("reverse", reverse_node)

# エントリポイントの設定
workflow.____("uppercase")

# エッジの追加 (直列接続)
workflow.____("uppercase", "reverse")
workflow.____("reverse", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [____(content="Hello LangGraph")]}
for s in app.____(inputs):
    print(s)

final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答002</summary>

``````python
# 解答002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。
*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>

### ■ 問題003: 条件付きエッジによる分岐の導入

LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

In [ ]:
# 解答欄003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

# エントリポイントの設定
workflow.____("check_number")

# 条件付きエッジの追加
workflow.____(
    "check_number", # 遷移元のノード
    ____,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
 )

# 各分岐からの終了エッジ
workflow.____("even_node", ____)
workflow.____("odd_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [____(content="42")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [____(content="77")]}
for s in app.____(inputs_odd):
    print(s)
final_state_odd = app.____(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")


<details><summary>解答003</summary>

``````python
# 解答003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

# エントリポイントの設定
workflow.set_entry_point("check_number")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check_number", # 遷移元のノード
    route_number,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
)

# 各分岐からの終了エッジ
workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
for s in app.stream(inputs_odd):
    print(s)
final_state_odd = app.invoke(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。
*   **コード解説:**
    *   `GraphState`に`number`という新しいキーを追加し、入力された数値を保持するようにしました。
    *   `check_number`ノードは、入力メッセージから数値を抽出し、`number`状態を更新します。
    *   `even_node`と`odd_node`は、それぞれ偶数と奇数だった場合の最終処理を行います。
    *   `route_number`関数がルーターとして機能します。この関数は現在の`state`を受け取り、次に実行すべきノードの名前（文字列）を返します。LangGraphは、この戻り値に基づいて適切なエッジを辿ります。
    *   `workflow.add_conditional_edges("check_number", route_number, {"even_node": "even_node", "odd_node": "odd_node"})`は、`check_number`ノードの後に`route_number`関数を実行し、その戻り値が`"even_node"`なら`even_node`へ、`"odd_node"`なら`odd_node`へ遷移するように設定しています。
---
</details>

### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。

In [ ]:
# 問題004: LLMに質問する

# ノートブックの先頭で初期化された共通のllmクライアントを使用します。
# LLM_PROVIDER の設定に応じて、異なるLLMが利用されます。

question = "LangGraphとは何ですか？簡単に説明してください。"

# 共通のllm変数を使ってLLMを呼び出します
try:
    response = llm.invoke(question)
    print(response.content)
except Exception as e:
    print(f"LLMの呼び出し中にエラーが発生しました: {e}")
    print("ノートブック冒頭のLLMプロバイダーの設定やAPIキーが正しく行われているか確認してください。")
    if LLM_PROVIDER == "google":
        print("Vertex AI をご利用の場合、Project ID と Location が正しく設定されているか、認証が完了しているか確認してください。")
    elif LLM_PROVIDER == "azure":
        print("Azure OpenAI をご利用の場合、Endpoint, API Key, API Version, Deployment Name が正しく設定されているか確認してください。")
    elif LLM_PROVIDER == "bedrock":
        print("Amazon Bedrock をご利用の場合、AWS認証情報（Access Key, Secret Key, Region）が正しく設定されているか確認してください。")


<details><summary>解答004</summary>

``````python
# 問題004: LLMに質問する

# ノートブックの先頭で初期化された共通のllmクライアントを使用します。
# LLM_PROVIDER の設定に応じて、異なるLLMが利用されます。

question = "LangGraphとは何ですか？簡単に説明してください。"

# 共通のllm変数を使ってLLMを呼び出します
try:
    response = llm.invoke(question)
    print(response.content)
except Exception as e:
    print(f"LLMの呼び出し中にエラーが発生しました: {e}")
    print("ノートブック冒頭のLLMプロバイダーの設定やAPIキーが正しく行われているか確認してください。")
    if LLM_PROVIDER == "google":
        print("Vertex AI をご利用の場合、Project ID と Location が正しく設定されているか、認証が完了しているか確認してください。")
    elif LLM_PROVIDER == "azure":
        print("Azure OpenAI をご利用の場合、Endpoint, API Key, API Version, Deployment Name が正しく設定されているか確認してください。")
    elif LLM_PROVIDER == "bedrock":
        print("Amazon Bedrock をご利用の場合、AWS認証情報（Access Key, Secret Key, Region）が正しく設定されているか確認してください。")
``````
</details>

<details><summary>解説004</summary>

このノートブックでは、様々なLLMプラットフォーム（OpenAI, Azure OpenAI, Google Vertex AI, Anthropic Claude, Amazon Bedrockなど）を簡単に切り替えて試せるように設計されています。
ノートブックの冒頭にある `LLM_PROVIDER` 変数で使用したいLLMを選択し、対応するAPIキーや環境変数を設定するだけで、この問題を含む全てのLLM呼び出し箇所で選択したLLMが利用されます。

ここでは、ノートブックの先頭で設定・初期化された共通の `llm` 変数を使用して、LLMに質問をしています。
`llm.invoke()` という統一されたインターフェースで、どのLLMプロバイダーを利用しているかに関わらず、同じようにLLMを呼び出すことができます。
これにより、特定のLLMサービスに依存しない、より汎用的なコードを作成するメリットを手軽に体験できます。

もしエラーが発生した場合は、ノートブック冒頭の `LLM_PROVIDER` の設定、および選択したプロバイダーに応じたAPIキーや環境変数の設定が正しく行われているかを確認してください。
各プロバイダー固有の設定項目（例えばVertex AIのProject ID、AzureのDeployment Nameなど）も見直してください。
---
</details>


### ■ 問題005: グラフの可視化とデバッグ

構築したLangGraphグラフの構造を視覚的に確認し、デバッグに役立てる方法を学びましょう。ここでは、これまでに作成したグラフのいずれか（例：問題003の条件分岐グラフ）を可視化し、その構造を理解します。

In [ ]:
# 解答欄005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

workflow.____("check_number")

workflow.____(
    "check_number",
    ____,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.____("even_node", ____)
workflow.____("odd_node", ____)

app = workflow.____()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    ____(____(app.____().____()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [____(content="10")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State: {final_state_even}")


<details><summary>解答005</summary>

``````python
# 解答005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

workflow.set_entry_point("check_number")

workflow.add_conditional_edges(
    "check_number",
    route_number,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

app = workflow.compile()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    display(Image(app.get_graph().draw_png()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [HumanMessage(content="10")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State: {final_state_even}")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント
*   **学習内容:** `app.get_graph().draw_png()`を使用してLangGraphのグラフ構造を画像として可視化する方法を学びます。これにより、複雑なグラフのデバッグや理解が容易になります。
*   **コード解説:**
    *   この問題では、問題003で作成した条件分岐グラフを再利用しています。これは、可視化の有用性を示すのに適した例だからです。
    *   `app.get_graph()`は、コンパイルされたグラフの内部表現を取得します。
    *   `.draw_png()`メソッドは、そのグラフ構造をPNG画像としてバイト列で返します。この機能を利用するには、システムにGraphvizがインストールされている必要があります。また、Pythonの`pygraphviz`や`pydotplus`といったライブラリも必要になる場合があります。
    *   `IPython.display.Image`と`display`を使うことで、Jupyter Notebook内で直接画像をレンダリングして表示できます。
    *   `try-except`ブロックでGraphvizのインストール状況によるエラーをハンドリングし、ユーザーに適切なメッセージを表示するようにしています。グラフが複雑になるにつれて、この可視化機能はデバッグや設計の確認に不可欠となります。
---
</details>